In [6]:
%pip install langchain langchain-community langchain_google_genai chromadb
%pip install  pypdf pandas streamlit python-dotenv 


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain in c:\users\testm\appdata\local\programs\python\python310\lib\site-packages (0.3.14)



You should consider upgrading via the 'c:\Users\testm\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\testm\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import Chroma 
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field 

import os
import pandas as pd
import streamlit as st
import tempfile 
from dotenv import load_dotenv 



c:\Users\testm\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\testm\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
load_dotenv()
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")


Define LLM 

In [9]:
llm = ChatGoogleGenerativeAI(model ="gemini-1.5-flash", google_api_key =GEMINI_API_KEY )


In [10]:
llm.invoke("Who is PM of India")


AIMessage(content='The current Prime Minister of India is Narendra Modi.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-c587d72d-0e91-431c-bed1-0046ca5137aa-0', usage_metadata={'input_tokens': 6, 'output_tokens': 11, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})

## Processing the Pdf

Load pdf document 



In [12]:
loader = PyPDFLoader("data/Coverage_Letter.pdf")
pages = loader.load()
pages

[Document(metadata={'source': 'data/Coverage_Letter.pdf', 'page': 0}, page_content='December 18, 2024\nTanmay vijay Pawar\n1b 1 Smith St\nBoston, Massachusetts 02120 \nTo Whom It May Concern: \nThis letter is to verify that Tanmay vijay Pawar, Insurance ID# NUQ 962586735, is insured under \nthe 2024-2025 Northeastern University Student Health Insurance Plan Student Health \nInsurance, Policy number 4955301. The effective dates of coverage are listed in the table below \nand may be subject to change. \nInsured’s Name Effective Date Termination Date\nTanmay vijay Pawar September 1, 2024 August 31, 2025\nThe 2024-2025 Northeastern University Student Health Insurance Plan Student Health \nInsurance Plan, policy number 4955301, provides worldwide coverage of eligible medical \nexpenses. Emergency Medical Evacuation and Repatriation of Remains are covered through  to \nan unlimited benefit maximum. Worldwide travel assistance is also available. Coverage is \nsubject to the terms and conditio

since the document can contain a lot of text content so it is good to split the text in chunks and paragraphs so that the chunk containing the required inforamation will be more focused.

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =200, 
                                               chunk_overlap =50, 
                                            #    length_funtion = len,
                                                 separators=["\n\n","\n"," "])
chunks =  text_splitter.split_documents(pages)

## Text Embeddings 
representing text or characters as numerical vectors.
These are hunge list of numbers 
similar pieces of text will have vectors close to each other whereas the texts which are not similar will have more distance in their vectors 
Distance between these vectors can be calculated using cosin similarity or eucladian distance 

## Create Embedding 

In [14]:
def get_embedding() :
    embeddings = GoogleGenerativeAIEmbeddings(
        model = "models/embedding-001", google_api_key=GEMINI_API_KEY  
    )
    return embeddings
embedding_function = get_embedding()
test_vector = embedding_function.embed_query("cat")


In [15]:
from langchain.evaluation import load_evaluator
evaluator = load_evaluator(evaluator="embedding_distance",
                           embeddings = embedding_function)

evaluator.evaluate_strings(prediction="water", reference="phone")



{'score': 0.07231365831958902}

## Vector Database

Create a new Chroma database from documents

** If vector embeddings for same document is created twice then chroma will save these as 2 separate chunks,, therfore db will contain duplicate data  **

In [16]:
import uuid 

def create_vectorstore(chunks, embedding_function, vectorstore_path): 

    #Create list of unique ids for each documnet based on content 
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks] 

    #Ensuring only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []

    for chunk , id in zip(chunks, ids ):
        if id not in unique_ids:
            unique_ids.add(id)
            unique_chunks.append(chunk)

    vectorstore =Chroma.from_documents(documents= unique_chunks,
                                       ids = list(unique_ids),
                                       embedding= embedding_function,
                                       persist_directory= vectorstore_path
                                       )
    vectorstore.persist()

    return vectorstore

In [17]:
vectorstore = create_vectorstore(chunks= chunks,
                                 embedding_function=  embedding_function,
                                 vectorstore_path= "vectorstore_chroma")

C:\Users\testm\AppData\Local\Temp\ipykernel_14156\3052451763.py:22: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


## Query relevant data

In [18]:
#Load vectorstore 
vectorestore = Chroma(persist_directory="vectorstore_chroma",
                      embedding_function= embedding_function)


C:\Users\testm\AppData\Local\Temp\ipykernel_14156\2627413748.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorestore = Chroma(persist_directory="vectorstore_chroma",


In [20]:
# Creater retreiver and get the relevant chunks 
retriever = vectorestore.as_retriever(search_type = "similarity")
relevant_chunks = retriever.invoke("what is date on document?")
relevant_chunks

[Document(metadata={'page': 0, 'source': 'data/Coverage_Letter.pdf'}, page_content='the 2024-2025 Northeastern University Student Health Insurance Plan Student Health \nInsurance, Policy number 4955301. The effective dates of coverage are listed in the table below'),
 Document(metadata={'page': 0, 'source': 'data/Coverage_Letter.pdf'}, page_content='December 18, 2024\nTanmay vijay Pawar\n1b 1 Smith St\nBoston, Massachusetts 02120 \nTo Whom It May Concern:'),
 Document(metadata={'page': 0, 'source': 'data/Coverage_Letter.pdf'}, page_content='and may be subject to change. \nInsured’s Name Effective Date Termination Date\nTanmay vijay Pawar September 1, 2024 August 31, 2025'),
 Document(metadata={'page': 0, 'source': 'data/Coverage_Letter.pdf'}, page_content='in order for payment to be considered.\nThe policy is underwritten by Blue Cross Blue Shield of MA and is serviced by Gallagher Student')]

In [21]:
PROMPT_TEMPLATE = """ You are a assitant for answering questions asked.
us the context given for reference and answer accordingly. If you don't know 
the answer just say you don't know. Do not make up any answers.

{context}
----
Answer the following question based on the above context : {question}
"""


In [25]:
#Combine the context text chunks
context_text = "\n\n----\n\n".join([doc.page_content for doc in relevant_chunks])

#Create prompt 
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context = context_text, question ="what is date on document?")

print(prompt)

Human:  You are a assitant for answering questions asked.
us the context given for reference and answer accordingly. If you don't know 
the answer just say you don't know. Do not make up any answers.

the 2024-2025 Northeastern University Student Health Insurance Plan Student Health 
Insurance, Policy number 4955301. The effective dates of coverage are listed in the table below

----

December 18, 2024
Tanmay vijay Pawar
1b 1 Smith St
Boston, Massachusetts 02120 
To Whom It May Concern:

----

and may be subject to change. 
Insured’s Name Effective Date Termination Date
Tanmay vijay Pawar September 1, 2024 August 31, 2025

----

in order for payment to be considered.
The policy is underwritten by Blue Cross Blue Shield of MA and is serviced by Gallagher Student
----
Answer the following question based on the above context : what is date on document?



### Generate Response

In [26]:
llm.invoke(prompt)

AIMessage(content='December 18, 2024', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a5b1fe16-b0f3-4160-bf07-116e4d91ccef-0', usage_metadata={'input_tokens': 238, 'output_tokens': 11, 'total_tokens': 249, 'input_token_details': {'cache_read': 0}})

### Using Lanchain expression language

In [33]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs
    ])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm 

)

rag_chain.invoke("What is date of document?")

AIMessage(content='December 18, 2024', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-6579b14d-d334-4c2d-908d-ea6fff9d7494-0', usage_metadata={'input_tokens': 280, 'output_tokens': 11, 'total_tokens': 291, 'input_token_details': {'cache_read': 0}})

### Generating structured response

In [55]:
class ExtractedInfo(BaseModel) :
    """ Extracted info from given pdf """
    document_date: str = Field(description="What is date of the document?")
    letter_for: str = Field(description="To whom the document belong?")
    document_summary: str = Field(description="Give summary of the document without any id's.")
    insurance_id : str = Field("what is document id of the user ?")


In [56]:

def ragChain(question):
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt_template
        | llm.with_structured_output(ExtractedInfo)
    )

    return rag_chain.invoke(question)


print(ragChain("Give me name of the provider,document date, owner name, summary, document id"))

document_date='September 1, 2024' letter_for='Tanmay vijay Pawar' document_summary='This letter is to verify that Tanmay vijay Pawar is insured under CVS Caremark. Coverage is subject to the terms and conditions of the policy.' insurance_id='NUQ 962586735'
